In [1]:
import os
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import dlib

## Prerequisites: Merekam Video
> Ajaklah teman / keluarga anda untuk membantu dalam proses perekaman ini

1. Rekamlah sebuah video yang berdurasi kira-kira 60 detik dengan kamera ponsel. Toleransi durasi adalah 63 detik.
2. Resolusi video haruslah tepat 1920x1080 dengan FPS 30.
3. Gunakan tripod / phone holder agar video tetap stabil
4. Lihat contoh gambar untuk posisi framing yang benar

<img src="data/sample_toby.jpg" alt="Contoh Framing" style="width:30%;">

5. Pastikan merekam dalam kondisi pencahayaan yang cukup (cahaya alami matahari lebih disarankan)
6. Anda harus duduk dengan tegak dan bersandar di kursi dan berada di tengah frame. Letakkan kedua tangan di atas lutut anda.
7. Selama perekaman, mintalah teman / keluarga anda untuk menghitung berapa kali anda bernafas selama 60 detik. Bukan hanya sekedar menghitung, teman / keluarga anda harus mencatat pada detik keberapa anda mulai menghembuskan nafas. Oleh karena itu, pastikan teman / keluarga anda menggunakan stopwatch pada HP mereka. (Tekan `LAP` setiap kali anda mulai menghembuskan nafas)
8. Pindahkan catatan tersebut ke dalam `csv` dengan format sebagai berikut:
```
| nafas-ke | detik | milidetik |
| -------- | ----- | --------- |
| 3        | 0     | 234       |
| 2        | 1     | 897       |
| dst      | dst   | dst       |
|          |       |           |
```

## Image vs Video

### Membuat video dari gambar (image sequence)

Definisikan lokasi direktori dari gambar yang akan dijadikan video.

In [8]:
IMGS_PATH = (os.path.join(os.getcwd(), 'data', 'toby-rgb'))
list_imgs = glob(IMGS_PATH + '/*.jpg')
print(f"5 path pertama: {list_imgs[:5]}")

5 path pertama: ['/Users/martinmanullang/Developer/if4021-handson/data/toby-rgb/1724_20230130_092547_475115.jpg', '/Users/martinmanullang/Developer/if4021-handson/data/toby-rgb/446_20230130_092504_883054.jpg', '/Users/martinmanullang/Developer/if4021-handson/data/toby-rgb/507_20230130_092506_915114.jpg', '/Users/martinmanullang/Developer/if4021-handson/data/toby-rgb/1682_20230130_092546_083114.jpg', '/Users/martinmanullang/Developer/if4021-handson/data/toby-rgb/1783_20230130_092549_451324.jpg']


Jangan lupa, jika bekerja dengan image sequence, pastikan urutan nama file sudah sesuai. Bila perlu, lakukan sorting

In [9]:
list_imgs = sorted(list_imgs, key=lambda x: int(x.split('/')[-1].split('.')[0]))
print(f"5 path pertama setelah diurutkan: {list_imgs[:5]}")
print(f"Total jumlah gambar: {len(list_imgs)}")

5 path pertama setelah diurutkan: ['/Users/martinmanullang/Developer/if4021-handson/data/toby-rgb/0_20230130_092450_022080.jpg', '/Users/martinmanullang/Developer/if4021-handson/data/toby-rgb/1_20230130_092450_051002.jpg', '/Users/martinmanullang/Developer/if4021-handson/data/toby-rgb/2_20230130_092450_083041.jpg', '/Users/martinmanullang/Developer/if4021-handson/data/toby-rgb/3_20230130_092450_115060.jpg', '/Users/martinmanullang/Developer/if4021-handson/data/toby-rgb/4_20230130_092450_147023.jpg']
Total jumlah gambar: 1800


**Tugas Hands-on 3**
1. Jelaskan maksud dari `list_imgs = sorted(list_imgs, key=lambda x: int(x.split('/')[-1].split('.')[0]))`

Memuat semua gambar yang ada di dalam `list_imgs` ke dalam numpy array. Setiap gambar akan diubah menjadi array 3 dimensi (RGB).

In [10]:
images = []
for img_path in list_imgs:
    img = cv2.imread(img_path)
    images.append(img)

images_array = np.array(images)

print(f"Shape of images_array: {images_array.shape}")

KeyboardInterrupt: 

Dapat kita amati bahwa ukuran dari `images_array` adalah berformat `(jumlah gambar, tinggi, lebar, channel)` atau umumnya ditulis `(t, h, w, c)`

Sekarang, mari kita buat sebuah video dari image sequence yang telah kita muat ke dalam `images_array`.

> Warning!
> Mungkin komputer Anda akan kehabisan memori jika jumlah gambar yang dijadikan video terlalu banyak. Jika hal ini terjadi, Anda bisa mengurangi jumlah gambar yang dijadikan video atau menggunakan komputer dengan spesifikasi yang lebih tinggi.

In [ ]:
save_loc = os.path.join(os.getcwd(), 'data', 'toby-rgb.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
height, width, layers = images_array[0].shape
video = cv2.VideoWriter(save_loc, fourcc, 1, (width, height))

for image in images_array:
    video.write(image)

video.release()

print("Video berhasil dibuat.")

**Tugas Hands-on 3**

2. Jelaskan tentang bagian kode berikut:
    ```python
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    ```

    Apakah ada opsi lain selain `mp4v`? Jika ada, coba gunakan dan jelaskan.

---
### Menyimpan setiap frame pada video menjadi gambar

Setelah anda berhasil membuat video dari image sequence, sekarang kita akan mencoba menyimpan setiap frame pada video menjadi gambar.
Pada eksperimen anda sendiri di rumah, mungkin anda perlu menjalankan bagian ini terlebih dahulu apabila sumber data anda adalah video dan anda belum memiliki image sequence.

In [ ]:
VID_PATH = os.path.join(os.getcwd(), 'data', 'toby-rgb.mp4')
IMGSEQ_PATH = os.path.join(os.getcwd(), 'data', 'toby-rgb-imgseq')

if not os.path.exists(IMGSEQ_PATH):
    os.makedirs(IMGSEQ_PATH)

cap = cv2.VideoCapture(VID_PATH)
frame_count = 0

while frame_count < 100:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imwrite(os.path.join(IMGSEQ_PATH, f'frame_{frame_count:03d}.jpg'), frame)
    frame_count += 1

cap.release()
print("100 frame pertama berhasil disimpan.")

**Tugas Hands-on 3**

3. Membuat video dengan FPS yang lebih rendah.
    - Dengan menggunakan video, simpanlah frame gambar setiap 3 frame. Begini ilustrasinya, jika ada frame 1 s/d 30, maka anda hanya perlu menyimpan frame 1, 4, 7, 10, 13, 16, 19, 22, 25, 28.
    - Dengan analogi ini, artinya anda mengurangi FPS-nya. Berapakah FPS yang baru?
    - Untuk setiap gambar, convertlah ke dalam format grayscale dan resize menjadi 1280 x 720.
    - Untuk setiap gambar, berikanlah titik merah (ukuran bebas, namun terlihat ketika video diputar). Titik tersebut bergerak dari kiri ke kanan untuk setiap frame. Titik tersebut harus sampai di ujung kanan gambar pada frame terakhir. Anda harus melakukan ini secara manual dengan memanipulasi matriks (tidak boleh pakai fungsi / library yang sudah ada)
    - Ingat, karena ini titik merah, maka channel warna pada video anda haruslah RGB (walaupun gambarnya telah menjadi grayscale).
    - Save video tersebut dengan nama `video_low_fps.mp4`

## Deteksi Wajah dengan dlib

### Instalasi dlib

Silahkan lihat [panduan ini](instalasi_dlib.md)

### Memindahkan data video ke dalam numpy array

Pertama-tama kita akan memindahkan data video ke dalam numpy array. Kita akan menggunakan `cv2.VideoCapture` untuk membaca video.

In [ ]:
VID_PATH = os.path.join(os.getcwd(), 'data', 'toby-rgb.mp4')

cap = cv2.VideoCapture(VID_PATH)
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

frames_array = np.array(frames)
cap.release()

print(f"Shape of frames_array: {frames_array.shape}")

Ambil satu gambar sebagai contoh

In [ ]:
single_img = frames_array[0].copy()
plt.figure(figsize=(5, 5))
plt.imshow(single_img)
plt.axis('off')
plt.show()

Tampaknya kita perlu mengubah warna gambar dari BGR menjadi RGB.

In [ ]:
frames_array_rgb = np.array([cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) for frame in frames_array])
print(f"Shape of frames_array_rgb: {frames_array_rgb.shape}")

In [ ]:
single_img = frames_array_rgb[1].copy()
plt.figure(figsize=(5, 5))
plt.imshow(single_img)
plt.axis('off')
plt.show()

Deteksi dengan dlib

In [ ]:
detector = dlib.get_frontal_face_detector()

faces = detector(single_img, 1)
for i, face in enumerate(faces): # untuk setiap wajah yang terdeteksi (bisa saja lebih dari satu)
    x, y, w, h = face.left(), face.top(), face.width(), face.height()
    x2 = x + w
    y2 = y + h
    cv2.rectangle(single_img, (x, y), (x2, y2), (255, 0, 0), 5)

plt.figure(figsize=(5, 5))
plt.imshow(single_img)
plt.axis('off')
plt.show()

```python
faces = detector(single_img, 1)
```
- **`detector(single_img, 1)`**: Ini menggunakan `dlib` face detector (`detector`), yang akan mendeteksi wajah pada gambar `single_img`.
  - `single_img`: Ini adalah gambar input yang ingin dideteksi wajahnya.
  - `1`: Ini adalah parameter opsional yang menentukan jumlah piramida gambar yang digunakan oleh algoritma deteksi wajah. Nilai `1` berarti algoritma akan menggunakan piramida untuk mencari wajah dalam berbagai ukuran.
- **`faces`**: Variabel ini menyimpan hasil deteksi wajah. Ini adalah sebuah list yang berisi objek bounding box untuk setiap wajah yang ditemukan pada gambar. Jika ada lebih dari satu wajah, `faces` akan berisi beberapa item.

```python
for i, face in enumerate(faces):
```
- **`for i, face in enumerate(faces)`**: Ini adalah loop yang akan berjalan sebanyak jumlah wajah yang terdeteksi.
  - **`enumerate(faces)`**: `enumerate` memberikan dua nilai untuk setiap iterasi loop: 
    - `i`: Indeks dari wajah yang terdeteksi (dimulai dari 0).
    - `face`: Objek bounding box dari setiap wajah yang terdeteksi. Objek ini memiliki koordinat posisi wajah dalam gambar.

```python
x, y, w, h = face.left(), face.top(), face.width(), face.height()
```
- **`face.left()`**: Mengambil koordinat x dari tepi kiri kotak pembatas (bounding box) wajah.
- **`face.top()`**: Mengambil koordinat y dari tepi atas kotak pembatas wajah.
- **`face.width()`**: Mengambil lebar bounding box wajah.
- **`face.height()`**: Mengambil tinggi bounding box wajah.
- **`x, y, w, h`**: Variabel ini menyimpan posisi dan ukuran dari bounding box wajah yang terdeteksi, di mana:
  - `x`: Koordinat x dari sudut kiri atas wajah.
  - `y`: Koordinat y dari sudut kiri atas wajah.
  - `w`: Lebar bounding box wajah.
  - `h`: Tinggi bounding box wajah.

```python
x2 = x + w
```
- **`x2 = x + w`**: Menghitung koordinat x dari sudut kanan bawah bounding box. Ini didapat dengan menjumlahkan nilai `x` (koordinat kiri atas) dengan `w` (lebar bounding box).

```python
y2 = y + h
```
- **`y2 = y + h`**: Menghitung koordinat y dari sudut kanan bawah bounding box. Ini didapat dengan menjumlahkan nilai `y` (koordinat kiri atas) dengan `h` (tinggi bounding box).

```python
cv2.rectangle(single_img, (x, y), (x2, y2), (255, 0, 0), 5)
```
- **`cv2.rectangle(single_img, (x, y), (x2, y2), (255, 0, 0), 5)`**:
  - Ini menggunakan OpenCV (`cv2`) untuk menggambar kotak persegi panjang (bounding box) di sekitar wajah yang terdeteksi.
  - **`single_img`**: Gambar tempat kotak akan digambar.
  - **`(x, y)`**: Titik sudut kiri atas dari bounding box (posisi awal persegi).
  - **`(x2, y2)`**: Titik sudut kanan bawah dari bounding box (posisi akhir persegi).
  - **`(255, 0, 0)`**: Warna persegi panjang dalam format BGR (Biru, Hijau, Merah), di mana (255, 0, 0) berarti biru.
  - **`5`**: Ketebalan garis persegi panjang.

Jadi, baris ini menggambar kotak biru dengan ketebalan garis 5 pixel di sekitar wajah yang terdeteksi pada gambar.

Bagian rambut dari subject tidak tercover oleh bounding box, bagaimana menyesuaikannya? Yang perlu diubah adalah `y` dan `h`.

In [ ]:
single_img = frames_array_rgb[1].copy()
faces = detector(single_img, 1)
for i, face in enumerate(faces): # untuk setiap wajah yang terdeteksi (bisa saja lebih dari satu)
    x, y, w, h = face.left(), face.top(), face.width(), face.height()
    print(f"x,y,w,h: {x, y, w, h}")
    
    y_edit = int(y - (0.5 * y))
    h_edit = int(h + (0.5 * h))
    print(f"x,y_edit,w,h_edit: {x, y_edit, w, h_edit}")
    
    cv2.rectangle(single_img, (x, y_edit), (x2, y_edit + h_edit), (0, 255, 0), 5)

plt.figure(figsize=(5, 5))
plt.imshow(single_img)
plt.axis('off')
plt.show()

**Tugas Hands-on 3**

4. Berdasarkan ROI Wajah, sesuaikanlah ROI tersebut untuk menyeleksi area bahu hingga dada.

### Mendeteksi Wajah pada Video

In [2]:
VID_PATH = os.path.join(os.getcwd(), 'data', 'toby-rgb.mp4')
detector = dlib.get_frontal_face_detector()

cap = cv2.VideoCapture(VID_PATH)
fps = 30
delay = int(1000 / fps)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    ### PROSES SETIAP FRAME DISINI ###
    faces = detector(frame, 1)
    for i, face in enumerate(faces): # untuk setiap wajah yang terdeteksi (bisa saja lebih dari satu)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
    ### AKHIR DARI PEMROSESAN TIAP FRAME ###
    
    cv2.imshow('Video', frame)
    
    if cv2.waitKey(delay) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

2024-10-17 17:12:55.526 python[27777:789963] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-17 17:12:55.526 python[27777:789963] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


KeyboardInterrupt: 

: 

---
## Catatan
- Dikarenakan banyak file yang berukuran besar seperti video dan gambar, tampaknya beberapa orang akan mengalami kendala ketika mensinkronkan dengan github
- Oleh karena itu, file video dan gambar tidak wajib diupload ke github. Anda dapat melakukan upload ke google drive dan memuatnya di Jupyter Notebook. Anda dapat menggunakan library `gdown`. [Cek tutorial berikut](https://www.youtube.com/watch?v=tKM95YWBc0o&t=630s).